In [2]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
from torchtext import data, datasets

ModuleNotFoundError: No module named 'torchtext'

In [12]:
pprint = lambda st : print(st.expandtabs(32))

In [27]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

N_BATCH = 64
lr = 0.001
N_EPOCH = 5

In [3]:
TEXT = data.Field(sequential=True, batch_first=True, lower=True)
LABEL = data.Field(sequential=False, batch_first=True)

train_ds, test_ds = datasets.IMDB.splits(TEXT, LABEL)

In [24]:
TEXT.build_vocab(train_ds, min_freq=5)             
LABEL.build_vocab(train_ds)

n_vocab = len(TEXT.vocab)
n_class = len(LABEL.vocab)
pprint(f'# vocabs train \t {n_vocab}')             # TEXT.vocab.stoi -> word2id
pprint(f'# classes \t {len(LABEL.vocab)}')       

# vocabs train                   25221
# classes                        3


In [19]:
train_ds, valid_ds = train_ds.split(split_ratio=0.8)
train_it, val_it, test_it = data.BucketIterator.splits((train_ds, valid_ds, test_ds), batch_size=N_BATCH, shuffle=True)

In [36]:
class GRU(nn.Module):
    def __init__(self, n_layers, n_hidden, n_vocab, n_embed, n_classes):
        super(GRU, self).__init__()
        self.n_layers = n_layers
        self.n_hidden = n_hidden

        self.embed = nn.Embedding(n_vocab, n_embed)
        self.dropout = nn.Dropout(0.2)
        self.gru = nn.GRU(n_embed, self.n_hidden, num_layers=self.n_layers, batch_first=True)
        self.out = nn.Linear(self.n_hidden, n_classes)

    def forward(self, x):
        x = self.embed(x)
        h_0 = self.init(n_batch=x.size(0))      # init first hidden to 0
        x, _ = self.gru(x, h_0)                 # (n_batch, n_seq, n_hidden)
        h_t = x[:,-1,:]                         # resize to (n_batch, Hidden)
        self.dropout(h_t)
     
        return self.out(h_t)                    # (n_batch, n_hidden) -> (n_batch, n_out)

    def init(self, n_batch=1):
        weight = next(self.parameters()).data
        return weight.new(self.n_layers, n_batch, self.n_hidden).zero_()

In [49]:
model = GRU(1, 32, n_vocab, 16, n_class).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

print(f"Total params {sum(p.numel() for p in model.parameters() if p.requires_grad)}")

Total params 408435


In [40]:
def train(model, optimizer, train_iter):
    model.train()
    for b, batch in enumerate(train_iter):
        x, y = batch.text.to(device), batch.label.to(device)
        y.data.sub_(1)  # 레이블 값을 0과 1로 변환
        optimizer.zero_grad()

        logit = model(x)
        loss = F.cross_entropy(logit, y)
        loss.backward()
        optimizer.step()

def evaluate(model, val_iter):
    """evaluate model"""
    model.eval()
    corrects, total_loss = 0, 0
    for batch in val_iter:
        x, y = batch.text.to(device), batch.label.to(device)
        y.data.sub_(1) # 레이블 값을 0과 1로 변환
        logit = model(x)
        loss = F.cross_entropy(logit, y, reduction='sum')
        total_loss += loss.item()
        corrects += (logit.max(1)[1].view(y.size()).data == y.data).sum()
    size = len(val_iter.dataset)
    avg_loss = total_loss / size
    avg_accuracy = 100.0 * corrects / size
    return avg_loss, avg_accuracy

best_val_loss = None
for e in range(1, N_EPOCH+1):
    train(model, optimizer, train_it)
    val_loss, val_accuracy = evaluate(model, val_it)

    print("[Epoch: %d] val loss : %5.2f | val accuracy : %5.2f" % (e, val_loss, val_accuracy))

    # 검증 오차가 가장 적은 최적의 모델을 저장
    if not best_val_loss or val_loss < best_val_loss:
        if not os.path.isdir("snapshot"):
            os.makedirs("snapshot")
        torch.save(model.state_dict(), './snapshot/txtclassification.pt')
        best_val_loss = val_loss

[Epoch: 1] val loss :  0.72 | val accuracy : 48.91


KeyboardInterrupt: 